In [1]:
import corpus
import os

In [2]:
import CaboCha

In [3]:
c = CaboCha.Parser()
txt = '明日は暑いし湿度も高いので、部活は休みにして早めに帰りましょう。'
#txt = 'そうしたらそのワンちゃんがなんかか喜んじゃって、で、あたしの方に走ってきて、とびついてきちゃってさ。'
tree = c.parse(txt)
print(tree.toString(CaboCha.FORMAT_LATTICE))

* 0 1D 0/1 0.516644
明日	名詞,副詞可能,*,*,*,*,明日,アシタ,アシタ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 1 3D 0/1 1.189807
暑い	形容詞,自立,*,*,形容詞・アウオ段,基本形,暑い,アツイ,アツイ
し	助詞,接続助詞,*,*,*,*,し,シ,シ
* 2 3D 0/1 2.251458
湿度	名詞,一般,*,*,*,*,湿度,シツド,シツド
も	助詞,係助詞,*,*,*,*,も,モ,モ
* 3 8D 0/1 -1.415575
高い	形容詞,自立,*,*,形容詞・アウオ段,基本形,高い,タカイ,タカイ
ので	助詞,接続助詞,*,*,*,*,ので,ノデ,ノデ
、	記号,読点,*,*,*,*,、,、,、
* 4 8D 0/1 -1.415575
部活	名詞,一般,*,*,*,*,部活,ブカツ,ブカツ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 5 6D 0/1 2.162031
休み	名詞,一般,*,*,*,*,休み,ヤスミ,ヤスミ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
* 6 8D 0/1 -1.415575
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
* 7 8D 0/1 -1.415575
早め	名詞,一般,*,*,*,*,早め,ハヤメ,ハヤメ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
* 8 -1D 0/2 0.000000
帰り	動詞,自立,*,*,五段・ラ行,連用形,帰る,カエリ,カエリ
ましょ	助動詞,*,*,*,特殊・マス,未然ウ接続,ます,マショ,マショ
う	助動詞,*,*,*,不変化型,基本形,う,ウ,ウ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [4]:
id = 0
for i in range(tree.size()):
    token = tree.token(i)
    if(token.chunk):
        tmp = token.chunk
        print(id, tmp.link, tmp.head_pos, tmp.func_pos, token.surface)
        id += 1

0 1 0 1 明日
1 3 0 1 暑い
2 3 0 1 湿度
3 8 0 1 高い
4 8 0 1 部活
5 6 0 1 休み
6 8 0 1 し
7 8 0 1 早め
8 -1 0 2 帰り


In [5]:
def bunsetsu_list(tree):
    result = []
    chunkindex = 0
    for i in range(tree.size()):
        token = tree.token(i)
        if(token.chunk is not None):
            features = token.feature.split(',')
            result.append({'lemma':features[-3], 'link':token.chunk.link, 'index':chunkindex})
            chunkindex += 1
    return result

def search_candidate(known_word_index, blist):
    result = set()
    for b in blist:
        if((b['link'] == known_word_index) or (b['index'] == blist[known_word_index]['link'])):
            result.add(b['index'])
    return sorted(list(result))

In [6]:
cp = corpus.Corpus(os.path.join('moddata','nucc','data001.txt'))

In [7]:
dic = dict()
with open(os.path.join('seeds', 'positive.txt')) as f:
    for w in f.readlines():
        sw = w.strip()
        dic[sw] = corpus.WordDicElement(sw)
        dic[sw].set_value(1.)
with open(os.path.join('seeds', 'negative.txt')) as f:
    for w in f.readlines():
        sw = w.strip()
        dic[sw] = corpus.WordDicElement(sw)
        dic[sw].set_value(-1.)


In [8]:
dic

{'喜ぶ': <corpus.WordDicElement at 0x1118013d0>,
 '嬉しい': <corpus.WordDicElement at 0x111ae9100>,
 '楽しい': <corpus.WordDicElement at 0x111ae97c0>,
 '美味しい': <corpus.WordDicElement at 0x111ae9d90>,
 'きれい': <corpus.WordDicElement at 0x111ae9820>,
 '美しい': <corpus.WordDicElement at 0x1114e0dc0>,
 '楽しむ': <corpus.WordDicElement at 0x1114e03d0>,
 '悲しい': <corpus.WordDicElement at 0x111ae98e0>,
 'さみしい': <corpus.WordDicElement at 0x111ae9e50>,
 '辛い': <corpus.WordDicElement at 0x111ae9a30>,
 '寂しい': <corpus.WordDicElement at 0x111ae9850>,
 'きたない': <corpus.WordDicElement at 0x111ae98b0>,
 '痛い': <corpus.WordDicElement at 0x111ad49d0>,
 'まずい': <corpus.WordDicElement at 0x111ad4400>,
 '残念': <corpus.WordDicElement at 0x111ad44f0>}

In [9]:
dic['喜ぶ']

In [10]:
def calc_score(cp, argdic):
    dic = argdic.copy()
    linecount = 0
    for line in cp.conversation:
        linecount += 1
        print(f'  {linecount}')
        for bindex, b in enumerate(line.bunsetsu_index_list):
            bunsetsu_head = line.sentence_word_list[b]
            try:
                dic[bunsetsu_head.lemma].accesscount += 1
                if(dic[bunsetsu_head.lemma].isvisited and abs(dic[bunsetsu_head.lemma].value) > 0.7):
                    candidates = [line.bunsetsu_index_list[i] for i in line.search_candidate(bindex)]
                    for candidate_index in candidates:
                        candidate = line.sentence_word_list[candidate_index]
                        try:
                            dic[candidate.lemma].add_score(corpus.extract(dic[bunsetsu_head.lemma], dic[candidate.lemma]))
                        except KeyError:
                            newword = corpus.WordDicElement(candidate.lemma)
                            newword.add_score(corpus.extract(dic[bunsetsu_head.lemma], newword))
                            dic[candidate.lemma] = newword
            except KeyError:
                continue
    return dic

def calc_value(argdic, alpha=0.5):
    dic = argdic.copy()
    for key in dic.keys():
        elem = dic[key]
        if(len(elem.score) < 1):
            continue
        elif(elem.isvisited):
            dic[key].set_value(sum(elem.score)/len(elem.score) * alpha + elem.value * (1-alpha))
        else:
            dic[key].set_value(sum(elem.score)/len(elem.score))
        dic[key].reset_score()
            
    return dic

In [11]:
#旧版
'''
def calc_score(cp, argdic):
    c = CaboCha.Parser()
    dic = argdic.copy()
    for line in cp.conversation:
        tree = c.parse(line['content'])
        blist = bunsetsu_list(tree)
        for bindex, b in enumerate(blist):
            try:
                dic[b['lemma']].accesscount += 1
                if(dic[b['lemma']].isvisited and abs(dic[b['lemma']].value) > 0.7):
                    candidates = search_candidate(bindex, blist)
                    for candidate_index in candidates:
                        candidate = blist[candidate_index]
                        try:
                            dic[candidate['lemma']].add_score(corpus.extract(dic[b['lemma']], dic[candidate['lemma']]))
                        except KeyError:
                            newword = corpus.WordDicElement(candidate.lemma)
                            newword.add_score(corpus.extract(dic[b['lemma']], newword))
                            dic[candidate['lemma']] = newword
            except KeyError:
                continue
    return dic

def calc_value(argdic, alpha=0.5):
    dic = argdic.copy()
    for key in dic.keys():
        elem = dic[key]
        if(len(elem.score) < 1):
            continue
        elif(elem.isvisited):
            dic[key].set_value(sum(elem.score)/len(elem.score) * alpha + elem.value * (1-alpha))
        else:
            dic[key].set_value(sum(elem.score)/len(elem.score))
        dic[key].reset_score()
            
    return dic
        
'''

"\ndef calc_score(cp, argdic):\n    c = CaboCha.Parser()\n    dic = argdic.copy()\n    for line in cp.conversation:\n        tree = c.parse(line['content'])\n        blist = bunsetsu_list(tree)\n        for bindex, b in enumerate(blist):\n            try:\n                dic[b['lemma']].accesscount += 1\n                if(dic[b['lemma']].isvisited and abs(dic[b['lemma']].value) > 0.7):\n                    candidates = search_candidate(bindex, blist)\n                    for candidate_index in candidates:\n                        candidate = blist[candidate_index]\n                        try:\n                            dic[candidate['lemma']].add_score(corpus.extract(dic[b['lemma']], dic[candidate['lemma']]))\n                        except KeyError:\n                            newword = corpus.WordDicElement(candidate.lemma)\n                            newword.add_score(corpus.extract(dic[b['lemma']], newword))\n                            dic[candidate['lemma']] = newword\n   

In [12]:
filecount = 0
for file in sorted(os.listdir(os.path.join('moddata','nucc'))):    
    filecount += 1
    print(filecount)
    cp = corpus.Corpus(os.path.join('moddata','nucc',file))
    dic = calc_score(cp, dic)
dic = calc_value(dic)

    

1
  1


AttributeError: 'Sentence' object has no attribute 'sentence_list'

In [ ]:
output = []
for i in dic.keys():
    if(-0.6 < dic[i].value < 0.6 or dic[i].accesscount > 1000):
        dic[i].set_value(0.)
        dic[i].deactivate()
        continue
    output.append(str(dic[i]))
with open('outputclassedfix1.txt', mode='w') as f:
    f.write('\n'.join(sorted(output, key=lambda x: int(x.split('(')[-1].replace(')','')), reverse=True)))